# Rollhammer

Is there a way to short circuit the Hit, Save, Wound sequence in Age of Sigmar? 

Here I've replaced the iterated roll with a single roll of several n-sided dice.

I've added an additional constraint that all the dice for the same (hit, save, wound) triple are of the same kind, mainly for aesthetics and simplicity.

In [1]:
import math
import bisect
import pandas as pd
from fractions import Fraction

In [2]:
def calc_p_wounds(to_hit, to_save, to_wound, attacks, wounds):
    """
    returns the probability of at least `wounds` made with `attacks`
    num attacks
    """
    p_wound = Fraction(7 - to_hit, 6) *\
              Fraction(7 - to_save, 6) *\
              Fraction(7 - to_wound, 6)
    p_miss = 1 - p_wound
    
    p_wounds = 0
    for exact_wounds in range(wounds, attacks+1):
        exact_misses = attacks - exact_wounds
        p_wounds += math.comb(attacks, exact_wounds)*\
                    (p_wound**exact_wounds)*\
                    (p_miss**exact_misses)
    return p_wounds

Achieving at least 1 wound from 3 attacks with a (3, 4, 5) (hit, save, wound) has a probability of ~29.77% (217/729)

In [3]:
calc_p_wounds(3, 4, 5, 3, 1)

Fraction(217, 729)

In [4]:
def calc_p_successes(dice, sides, successes, floor):
    """
    returns the probability of at least `successes` to have at least `floor`
    given `dice` dice with `sides` sides
    """
    p_success = Fraction(sides - floor + 1, sides)
    p_fail = 1 - p_success

    p_successes = 0
    for exact_success in range(successes, dice+1):
        exact_fail = dice - exact_success
        p_successes += math.comb(dice, exact_success)*\
                       (p_success**exact_success)*\
                       (p_fail**exact_fail)
    return p_successes

The probability of seeing at least 2 15+s when rolling 5d20 is ~47.18% (23,589/50,000)

In [5]:
calc_p_successes(5, 20, 2, 15)

Fraction(23589, 50000)

Precomputing all possible roll probabilities (up to 20 dice) can be increased for greater accuracy

I've added all the types of dice found in Tabletop Simulator

In [6]:
DICE_DF = (pd.DataFrame([(dice, sides, successes, floor,
                          calc_p_successes(dice, sides, successes, floor))
                         for dice in range(1, 21)
                         for sides in [4, 6, 8, 10, 12, 20]
                         for successes in range(1, dice+1)
                         for floor in range(1, sides+1)],
                        columns=["dice", "sides", "successes", "floor", "p_successes"])
           .sort_values(["dice", "sides", "p_successes", "floor", "successes"])
           .groupby(["dice", "sides", "p_successes"])
           .first())
print(len(DICE_DF))
DICE_DF[:3]

11452


successes  floor
dice sides p_successes                  
1    4     1/4                  1      4
           1/2                  1      3
           3/4                  1      2

In [7]:
def nearest_roll(dice_df, dice, sides, to_hit, to_save, to_wound, attacks, wounds):
    """
    returns the nearest (successes, floor, p_successes, p_wounds, squared_error) 
    for a given combination of attacks and wounds
    """
    p_wounds = calc_p_wounds(to_hit, to_save, to_wound, attacks, wounds)
    min_loc = dice_df.loc[dice, sides].index.get_loc(p_wounds, method="nearest")
    min_row = dice_df.loc[dice, sides].iloc[min_loc]
    se = (p_wounds - min_row.name)**2
    return (min_row.successes, min_row.floor, min_row.name, p_wounds, se)

The probability of getting at least 1 wound with 3 attacks with (3, 4, 5) is 217/729 (29.77%)

The same as `calc_p_wounds(3, 4, 5, 3, 1)`

The nearest probability roll given 5d20 is at least 3, 13+ which is 992/3125 (31.74%)

The last element in the tuple is the squared error, (p_successes-p_wounds)^2

In [8]:
nearest_roll(DICE_DF, 5, 20, 3, 4, 5, 3, 1)

(3,
 13,
 Fraction(992, 3125),
 Fraction(217, 729),
 Fraction(2028871849, 5189853515625))

In [9]:
def gen_roll_df(DICE_DF, dice, sides, to_hit, to_save, to_wound, attacks):
    """
    returns a dataframe with (wounds, p_successes, p_wounds, squared_error) for
    every num_wounds possible given attacks
    """
    acc = []
    possible_wounds = range(attacks, 0, -1)
    for wounds in possible_wounds:
        acc.append(nearest_roll(DICE_DF, dice, sides, 
                                to_hit, to_save, to_wound, 
                                attacks, wounds))
    return (pd.DataFrame(acc, columns=["successes", "floor", "p_successes", "p_wounds", "squared_error"])
            .assign(wounds=possible_wounds))

Calculates the nearest roll for all possible wounds

In [10]:
gen_roll_df(DICE_DF, 5, 20, 3, 4, 5, 3)

,successes,floor,p_successes,p_wounds,squared_error,wounds
0,3,20,1853/1600000,1/729,62082200569/1360488960000000000,3
1,5,11,1/32,25/729,5041/544195584,2
2,3,13,992/3125,217/729,2028871849/5189853515625,1


In [11]:
def nearest_dice(dice_df, to_hit, to_save, to_wound, attacks):
    """
    returns (mean_squared_error, dice, sides) for the lowest
    mean_squared_error for a given combination of attacks and wounds
    and the values in dice_df
    """
    acc = []
    for dice in dice_df.index.levels[0]:
        for sides in dice_df.index.levels[1]:
            mse = gen_roll_df(DICE_DF, dice, sides, to_hit, to_save, to_wound, attacks)["squared_error"].mean()
            acc.append((mse, dice, sides))
    return min(acc)

Finds the dice, side combination with the lowest mean squared error across all wounds

In this case that's 19d20s

In [12]:
nearest_dice(DICE_DF, 3, 4, 5, 3)

(2.8150516761149614e-07, 19, 20)

### Putting it all Together

In [13]:
def gen_rollhammer(dice_df, to_hit, to_save, to_wound, attacks):
    """
    returns a table for the corresponding single roll to replace the iterated roll
    """
    (mse, dice, sides) = nearest_dice(dice_df, to_hit, to_save, to_wound, attacks)
    df = (gen_roll_df(dice_df, dice, sides, to_hit, to_save, to_wound, attacks)
          .assign(dice=dice, sides=sides)
          .assign(p_successes_perc=lambda df: (df["p_successes"].astype(float)*100).round(4))
          .assign(p_wounds_perc=lambda df: (df["p_wounds"].astype(float)*100).round(4))
          .assign(error_perc=lambda df: ((df["p_successes"]-df["p_wounds"])/df["p_wounds"]*100).astype(float).round(4))
          [["dice", "sides", "wounds", "successes", "floor", "p_wounds_perc", "p_successes_perc", "error_perc"]])
    return df

When making 3 attacks with a (3, 4, 5) (hit, save, wound) roll 19d20s

* If you see at least 17, 10+ that's 3 wounds
* If you see at least 13, 12+ that's 2 wounds
* If you see at least 14, 8+ that's 1 wound

Otherwise, you swing and you miss

In [14]:
gen_rollhammer(DICE_DF, 3, 4, 5, 3)

,dice,sides,wounds,successes,floor,p_wounds_perc,p_successes_perc,error_perc
0,19,20,3,17,10,0.1372,0.1528,11.4153
1,19,20,2,13,12,3.4294,3.4231,-0.1816
2,19,20,1,14,8,29.7668,29.6765,-0.3035


Use larger attacks for more impressive tables

In [15]:
gen_rollhammer(DICE_DF, 4, 4, 4, 10)

,dice,sides,wounds,successes,floor,p_wounds_perc,p_successes_perc,error_perc
0,10,8,10,10,8,0.0000,0.0000,0.0
1,10,8,9,9,8,0.0000,0.0000,0.0
2,10,8,8,8,8,0.0002,0.0002,0.0
3,10,8,7,7,8,0.0040,0.0040,0.0
4,10,8,6,6,8,0.0510,0.0510,0.0
5,10,8,5,5,8,0.4455,0.4455,0.0
6,10,8,4,4,8,2.7464,2.7464,0.0
7,10,8,3,3,8,11.9502,11.9502,0.0
8,10,8,2,2,8,36.1102,36.1102,0.0
9,10,8,1,1,8,73.6924,73.6924,0.0
